# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df.head()

,City,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed
0,Pevek,69.70,170.31,-0.81,91,29,7.34
1,Mataura,-46.19,168.86,43.30,89,100,4.52
2,New Norfolk,-42.78,147.06,43.61,87,60,2.44
3,Saskylakh,71.92,114.08,8.49,79,0,5.37
4,Yellowknife,62.46,-114.35,23.27,85,90,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Latitude","Longitude"]].astype(float)
weights = cities_df["Humidity"].astype(int)

In [6]:
center = cities_df[["Latitude","Longitude"]].median()
max_humidity = cities_df["Humidity"].astype(float).max()
fig = gmaps.figure(center=center, zoom_level=2)
humidity_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, 
                                     max_intensity=max_humidity, point_radius = 3)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
my_cities_df = cities_df.loc[(cities_df['Latitude'] > 20) &
                             (cities_df['Temperature (F)'] < 85) &
                             (cities_df['Temperature (F)'] > 70) &
                             (cities_df['Humidity'] < 70) &
                             (cities_df['Cloudiness'] < 5) &
                             (cities_df['Wind Speed'] < 12)]

my_cities_df = my_cities_df.dropna()
my_cities_df

,City,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed
34,Kapaa,22.08,-159.32,78.89,65,1,8.05
154,Makakilo City,21.35,-158.09,79.52,57,1,9.17
156,Blythe,33.61,-114.60,79.18,22,1,5.82
176,Tarabha,20.73,83.65,82.72,36,2,3.33
229,Hurghada,27.26,33.81,71.60,40,0,4.70
270,Barwāla,29.38,75.92,75.31,41,3,0.85
529,Pakokku,21.33,95.10,81.82,45,2,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotels_df = my_cities_df.copy()
hotels_df["Hotel Name"] = ""

In [9]:
params = {
    "location": "",
    "keyword":"hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotels_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    params["location"]=f"{latitude},{longitude}"
    response = requests.get(base_url, params=params).json()
    if len(response['results']) > 0:
        hotel_name = response['results'][0]['name']
    else:
        hotel_name = "No Hotel Found"
    hotels_df.loc[index, 'Hotel Name'] = hotel_name

In [10]:
hotels_df

,City,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed,Hotel Name
34,Kapaa,22.08,-159.32,78.89,65,1,8.05,Sheraton Kauai Resort at Coconut Beach
154,Makakilo City,21.35,-158.09,79.52,57,1,9.17,Marriott's Ko Olina Beach Club
156,Blythe,33.61,-114.60,79.18,22,1,5.82,Hampton Inn & Suites Blythe
176,Tarabha,20.73,83.65,82.72,36,2,3.33,No Hotel Found
229,Hurghada,27.26,33.81,71.60,40,0,4.70,MERAKI Resort
270,Barwāla,29.38,75.92,75.31,41,3,0.85,OYO 67609 Hotel Taj
529,Pakokku,21.33,95.10,81.82,45,2,2.30,Hotel Juno


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
hotel_locations = hotels_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
hotels_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(hotels_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))